In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime

# Simulated weather data (replace with your own dataset)
num_samples = 1000
sequence_length = 46
num_features = 5

# Generate synthetic weather data (replace with actual data loading)
weather_data = np.random.rand(num_samples, sequence_length, num_features)

# Split your dataset into training and validation sets
# Assuming you have a dataset named 'weather_data'
from sklearn import model_selection

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    weather_data[:, :-1, :],  # Input: all time steps except the last
    weather_data[:, -1:, :],   # Target: all time steps except the first
    test_size=0.2,  # Adjust the validation split as needed
    random_state=42  # Set a random seed for reproducibility
)


class MultiScaleAttentionTransformer(tf.keras.Model):
    def __init__(self, num_heads, d_model, d_ff, sequence_length, num_features):
        super(MultiScaleAttentionTransformer, self).__init__()
        self.sequence_length = sequence_length
        self.num_features = num_features
        self.encoder = tf.keras.Sequential([
            keras.layers.Input(shape=(None, num_features)),  # Allow dynamic batch size
            keras.layers.Conv1D(d_model, kernel_size=1, activation='relu'),
            keras.layers.LayerNormalization(epsilon=1e-6)
        ])

        self.transformer = keras.Sequential([
            keras.layers.MultiHeadAttention(
                num_heads=num_heads, 
                key_dim=d_model // num_heads,  # Adjust key dimension
                value_dim=num_features  # Specify value dimension
            ),
            keras.layers.Dropout(0.1),
            keras.layers.LayerNormalization(epsilon=1e-6),
            keras.layers.Conv1D(d_ff, kernel_size=1, activation='relu'),
            keras.layers.Dropout(0.1),
            keras.layers.LayerNormalization(epsilon=1e-6)
        ])


        self.decoder = keras.Sequential([
            keras.layers.Conv1D(num_features, kernel_size=1)
        ])

        self.downsampler = keras.Sequential([
            keras.layers.Conv1D(d_model, kernel_size=3, strides=2, padding='same')
        ])

        self.upsampler = keras.Sequential([
            keras.layers.Conv1D(d_model, kernel_size=3, padding='same')
        ])

    
    def call(self, x):
        # Encoder
        x_encoded = self.encoder(x)
        print(f"Encoder Output Shape: {x_encoded.shape}")
        
        # Multi-Head Attention
        x_transformed = self.transformer(x_encoded)
        print(f"Multi-Head Attention Output Shape: {x_transformed.shape}")
        
        # Decoder
        x_decoded = self.decoder(x_transformed)
        print(f"Decoder Output Shape: {x_decoded.shape}")
        
        # Multi-Scale Attention
        x_downsampled = self.downsampler(x_encoded)
        print(f"Downsampled Output Shape: {x_downsampled.shape}")
        x_upsampled = tf.image.resize(x_transformed, (self.sequence_length, self.num_features))
        print(f"Upsampled Output Shape: {x_upsampled.shape}")
        x_multi_scale_attention = x_decoded + x_downsampled + x_upsampled
        print(f"Multi-Scale Attention Output Shape: {x_multi_scale_attention.shape}")
        
        return x_multi_scale_attention

# Hyperparameters
num_heads = 8
d_model = 64
d_ff = 128

# Create the model
model = MultiScaleAttentionTransformer(num_heads, d_model, d_ff, sequence_length, num_features)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Prepare the training data
# x_train = weather_data[:, :-1, :]  # Input: all time steps except the last
# y_train = weather_data[:, 1:, :]   # Target: all time steps except the first

# Convert the training data to TensorFlow tensors
# x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
# y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

# Set up TensorBoard callback with log directory
logdir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model with the TensorBoard callback and validation data
model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val),  # Add validation data here
    callbacks=[tensorboard_callback]
)

# Use the trained model for weather forecasting
forecast = model.predict(x_train[:1])  # Make a forecast for the first sequence

print("Forecasted Weather:")
print(forecast)


Epoch 1/10
Encoder Output Shape: (32, 45, 64)


TypeError: in user code:

    File "/home/xianjia/Workspace/miniconda3/envs/uwb-lstm-env/lib/python3.11/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/xianjia/Workspace/miniconda3/envs/uwb-lstm-env/lib/python3.11/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/xianjia/Workspace/miniconda3/envs/uwb-lstm-env/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/xianjia/Workspace/miniconda3/envs/uwb-lstm-env/lib/python3.11/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/xianjia/Workspace/miniconda3/envs/uwb-lstm-env/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(fi